In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
import optuna
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_optimization_history
import warnings
warnings.filterwarnings('ignore')
# Load the data from the CSV file
btc = pd.read_csv('btc.csv')
data = btc['Close'].values.reshape(-1, 1)
# Normalize the data using MinMaxScaler
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

# Split the data into training and testing sets
train_data = data[:800]
test_data = data[800:]

# Define the window size for the LSTM model
window_size = 50

# Define the function to create input and output sequences
def create_sequences(data, window_size):
    X = []
    y = []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i])
        y.append(data[i])
    X = np.array(X)
    y = np.array(y)
    return X, y

# Create the input and output sequences for the training and testing data
X_train, y_train = create_sequences(train_data, window_size)
X_test, y_test = create_sequences(test_data, window_size)

# Define the LSTM model architecture
def create_lstm_model(trial):
    units = trial.suggest_categorical('units', [32, 64, 128])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    activation_function = trial.suggest_categorical('activation_function', ['tanh', 'sigmoid', 'relu'])
    regularization = None
    if trial.suggest_categorical('regularization', [False, True]):
        regularization = trial.suggest_loguniform('regularization_value', 1e-6, 1e-3)
        regularization = keras.regularizers.l2(regularization)
    model = keras.Sequential([
        keras.layers.LSTM(units=units, input_shape=(window_size, 1), activation=activation_function, kernel_regularizer=regularization),
        keras.layers.Dense(units=1)
    ])
    optimizer = keras.optimizers.Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Define the objective function for Optuna
def objective(trial):
    model = create_lstm_model(trial)
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=0)
    loss = model.evaluate(X_test, y_test)
    return loss

# Define the Optuna study
study = optuna.create_study(direction='minimize')

# Start the optimization process
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and validation loss
best_trial = study.best_trial
print(f'Best trial - Loss: {best_trial.value}, Params: {best_trial.params}')

[I 2023-03-13 15:20:47,269] A new study created in memory with name: no-name-785313bf-3bb1-4922-9f80-088ca38cf948


14/14 [==============================] - 0s 13ms/step - loss: 2.2331e-05


[I 2023-03-13 15:21:26,553] Trial 0 finished with value: 2.2330874344334006e-05 and parameters: {'units': 128, 'learning_rate': 0.00017976200115191646, 'activation_function': 'tanh', 'regularization': True, 'regularization_value': 2.7519709216254224e-06}. Best is trial 0 with value: 2.2330874344334006e-05.


14/14 [==============================] - 0s 5ms/step - loss: 2.1106e-06


[I 2023-03-13 15:21:46,440] Trial 1 finished with value: 2.1105872747284593e-06 and parameters: {'units': 32, 'learning_rate': 0.0661050839636368, 'activation_function': 'tanh', 'regularization': False}. Best is trial 1 with value: 2.1105872747284593e-06.


...

14/14 [==============================] - 0s 7ms/step - loss: 4.7020e-05


[I 2023-03-13 16:12:07,907] Trial 98 finished with value: 4.701971192844212e-05 and parameters: {'units': 64, 'learning_rate': 0.00808141273842341, 'activation_function': 'tanh', 'regularization': False}. Best is trial 5 with value: 6.520823490063776e-07.


14/14 [==============================] - 0s 13ms/step - loss: 6.5044e-05


[I 2023-03-13 16:12:48,297] Trial 99 finished with value: 6.504441989818588e-05 and parameters: {'units': 128, 'learning_rate': 0.0008252590475550379, 'activation_function': 'tanh', 'regularization': False}. Best is trial 5 with value: 6.520823490063776e-07.


Best trial - Loss: 6.520823490063776e-07, Params: {'units': 128, 'learning_rate': 0.0011852442575562424, 'activation_function': 'tanh', 'regularization': False}


In [5]:
plot_optimization_history(study)

In [6]:

# Plot the parameter importances

plot_optimization_history(study)

In [7]:
plot_param_importances(study)